**Task 07: Querying RDF(s)**

In [1]:
!pip install rdflib 
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2020-2021/master/Assignment4"

     |████████████████████████████████| 235kB 2.9MB/s 
     |████████████████████████████████| 51kB 4.3MB/s 


Leemos el fichero RDF de la forma que lo hemos venido haciendo

In [2]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS
g = Graph()
g.namespace_manager.bind('ns', Namespace("http://somewhere#"), override=False)
g.namespace_manager.bind('vcard', Namespace("http://www.w3.org/2001/vcard-rdf/3.0#"), override=False)
g.parse(github_storage+"/resources/example6.rdf", format="xml")

<Graph identifier=Nffbee18b791046868472e3afa0543b4f (<class 'rdflib.graph.Graph'>)>

**TASK 7.1: List all subclasses of "Person" with RDFLib and SPARQL**

In [5]:
#RDFLib
ns = Namespace("http://somewhere#")
for s, p, o in g.triples((None, RDFS.subClassOf, ns.Person)):
  for s2, p2, o2 in g.triples((None, RDFS.subClassOf, s)):
    print(s2)


#SPARQL
from rdflib.plugins.sparql import prepareQuery
query = prepareQuery('''
  #PREFIX ex: <https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2020-2021/master/Assignment4>
  SELECT DISTINCT ?x
  WHERE{
	  ?x (rdfs:subClassOf/rdfs:subClassOf*) ns:Person		      # normalmente sería "?x rdfs:subClassOf ex:Establishment", pero como queremos navegar no sólo por la primera línea de subclases, 
  }							                                            # sino que por toda la jerarquía, se le añade un "/subClassOf*"
  LIMIT 100
  ''',
  initNs = {"rdfs": RDFS, "ns": ns}
)
for i in g.query(query):
  print(i)

http://somewhere#PhDstudent
(rdflib.term.URIRef('http://somewhere#Researcher'),)
(rdflib.term.URIRef('http://somewhere#PhDstudent'),)


**TASK 7.2: List all individuals of "Person" with RDFLib and SPARQL (remember the subClasses)**


In [6]:
#RDFLib
for s, p, o in g.triples((None, RDF.type, ns.Person)):
  for s2, p2, o2 in g.triples((None, RDFS.subClassOf, s)):
    print(s2)


#SPARQL
from rdflib.plugins.sparql import prepareQuery
query = prepareQuery('''
  PREFIX ex: <https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2020-2021/master/Assignment4>
  SELECT DISTINCT ?x
  WHERE{
	  {?p (rdfs:subClassOf/rdfs:subClassOf*) ns:Person} UNION
    {?x rdf:type ?p} UNION
    {?x rdf:type ns:Person}
  }
  LIMIT 100
  ''',
  initNs = {"rdfs": RDFS, "ns": ns}
)
for i in g.query(query):
  print(i)

(rdflib.term.URIRef('http://somewhere#JaneSmith'),)
(rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given'),)
(rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family'),)
(rdflib.term.URIRef('http://somewhere#PhDstudent'),)
(rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN'),)
(rdflib.term.URIRef('http://somewhere#UPM'),)
(rdflib.term.URIRef('http://somewhere#University'),)
(rdflib.term.URIRef('http://somewhere#Researcher'),)
(rdflib.term.URIRef('http://somewhere#JohnSmith'),)
(rdflib.term.URIRef('http://somewhere#Person'),)
(rdflib.term.URIRef('http://somewhere#SaraJones'),)


**TASK 7.3: List all individuals of "Person" and all their properties including their class with RDFLib and SPARQL**


In [7]:
#RDFLib
for s, p, o in g.triples((None, RDF.type, ns.Person)):
  for s2, p2, o2 in g.triples((s2, None, None)):
    print(s2)
for s, p, o in g.triples((None, RDFS.subClassOf, ns.Person)):
  for s2, p2, o2 in g.triples((None, RDF.type, s2)):
    for s3, p3, o3 in g.triples((s3, None, None)):
      print(s3)


#SPARQL
from rdflib.plugins.sparql import prepareQuery
query = prepareQuery('''
  PREFIX ex: <https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2020-2021/master/Assignment4>
  SELECT DISTINCT ?x ?p
  WHERE{
  	{?p (rdfs:subClassOf/rdfs:subClassOf*) ns:Person} UNION
    {?x rdf:type ?p} UNION
    {?x rdf:type ns:Person} UNION
    {?x ?p ?o}
  }
  LIMIT 100
  ''',
  initNs = {"rdfs": RDFS, "ns": ns}
)
for i in g.query(query):
  print(i)

http://somewhere#PhDstudent
http://somewhere#PhDstudent
http://somewhere#PhDstudent
http://somewhere#PhDstudent
(None, rdflib.term.URIRef('http://somewhere#Researcher'))
(None, rdflib.term.URIRef('http://somewhere#PhDstudent'))
(rdflib.term.URIRef('http://somewhere#JaneSmith'), rdflib.term.URIRef('http://somewhere#Researcher'))
(rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#Datatype'))
(rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#Property'))
(rdflib.term.URIRef('http://somewhere#PhDstudent'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#Class'))
(rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#Datatype'))
(rdflib.term.URIRef('http://somewhere#UPM'), rdflib.term.URIRef('http://somewhere#University'))